<a href="https://colab.research.google.com/github/mokroke/my-first-repo/blob/ubdate-readme/NN_test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セッション切れ回避

```
function ClickConnect(){
  console.log("60sごとに再接続");
  document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect,1000*60);
```

In [ ]:
# # catboostのinstall
!pip install catboost
!pip install scikit-learn
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
import ipywidgets as widgets

In [ ]:
button = widgets.Button(description = 'Click me')
button

Button(description='Click me', style=ButtonStyle())

In [ ]:
from IPython.display import display
display(button)

Button(description='Click me', style=ButtonStyle())

In [ ]:
#clickしたらprintされるやつ
button = widgets.Button(description = 'Click me')

def on_click_callback(clicked_button):
  print('Clicked')

button.on_click(on_click_callback)
display(button)


Button(description='Click me', style=ButtonStyle())

# ドライブのマウント

In [ ]:
import sys
import os
from pathlib import Path

if 'google.colab' in sys.modules:  # colab環境
    # drive mount
    from google.colab import drive
    drive.mount('/content/drive/')

    print('githubのリポジトリのパスを入力: ')
    print('例： /content/drive/MyDrive/git/kaggle-ubiquant-market-prediction')
    WORKING_DIR = input()
    # working dir に移動
    os.chdir(WORKING_DIR)
    print("cwd:", os.getcwd())
    
elif 'kaggle_web_client' in sys.modules:  # kaggle環境
    pass
else: # ローカル
    os.chdir('..')

#諸田のdrive  (/content/drive/MyDrive/kaggle-ubiquant-market-prediction)

Mounted at /content/drive/
githubのリポジトリのパスを入力: 
例： /content/drive/MyDrive/git/kaggle-ubiquant-market-prediction
/content/drive/MyDrive/kaggle-ubiquant-market-prediction
cwd: /content/drive/MyDrive/kaggle-ubiquant-market-prediction


## 設定

In [ ]:
#自動でモジュールをリロードしてくれる
%load_ext autoreload
%autoreload 2

In [ ]:
import ubiquantlib

In [ ]:
BASE_NAME = 'test_nn' # configファイルの読み込み先, logの出力先、モデルの保存先の名前

In [ ]:
import json
#jsonファイルの作成

st = {
    "num_kfolds":5,
    "random_state":42,
    "params":{}
    }

path = "/content/drive/MyDrive/kaggle-ubiquant-market-prediction/configs/"
with open(path + f'{BASE_NAME}.json', 'w') as f:
    json.dump(st, f, ensure_ascii=False)

In [ ]:
# pring の代わりに logger を使うようにする

"""
logの出力先と、logの設定ファイルの読み込み先を指定
filenameを設定しない場合は時間でファイルが作成される
log_config_pathを指定しない場合は読み込み先は ./log_config.json
ただし、使う場合は一番最初に行う（configなどのloadの前）
"""
# ubiquantlib.logs.init_logging(filename=f"./logs/{BASE_NAME}.logs", log_config_path='./log_config.json') # log_config_pathは指定しなくてもOK
ubiquantlib.logs.init_logging(filename=f"./logs/{BASE_NAME}.logs", log_config_path='./log_config.json') # log_config_pathは指定しなくてもOK

# logger の使い方
from logging import getLogger

logger = getLogger('main') # 基本は 'main' というloggerを使う
logger.info('test console and file') # info 以上はコンソールとファイル出力
logger.debug('test file') # debugはファイル出力のみ。後で必要になった時に見たいが、手元で表示したくない時に使う



# config から設定を読み込み
config = ubiquantlib.settings.load_config(f'configs/{BASE_NAME}.json')

 init_logging [INFO]: ./log_config.json loaded
 <module> [INFO]: test console and file


In [ ]:
config

{'num_kfolds': 5, 'params': {}, 'random_state': 42}

In [ ]:
os.listdir('./data/input')

['ubiquant-parquet']

In [ ]:
# データの読み込みとメモリ削減
train = ubiquantlib.datasets.load_train_parquet(low_memory=True)

# debug用： ランダムでサンプリングして量を減らす
# train = train.sample(n=100, random_state=config['random_state']).sort_index().reset_index(drop=True)

train = ubiquantlib.datasets.reduce_mem_usage(train)
# train.head()

Memory usage of dataframe is 3642.99 MB


100%|██████████| 304/304 [02:24<00:00,  2.10it/s]

Memory usage after optimization is: 1915.96 MB
Decreased by 47.4%


In [ ]:
# debug用： ランダムでサンプリングして量を減らす
train_s = train.sample(n=100, random_state=config['random_state']).sort_index().reset_index(drop=True)


In [ ]:
# train['time_id'].value_counts()

In [ ]:
# import numpy as np

# tscv = ubiquantlib.cv.GroupTimeSeriesSplit(n_splits=config['num_kfolds'])
# for fi,(train_index, valid_index) in enumerate(tscv.split(train, groups=train.time_id)):
#   print(f'fold {fi}')
#   print(f'train {len(train_index)}件:' , np.min(train_index), '~' ,np.max(train_index))
#   print(f'valid {len(valid_index)}件:' , np.min(valid_index), '~' ,np.max(valid_index))
#   print()

In [ ]:
# from sklearn.model_selection import TimeSeriesSplit

# tscv = TimeSeriesSplit(n_splits=config['num_kfolds'])
# for fi,(train_index, valid_index) in enumerate(tscv.split(train, groups=train.time_id)):
#   print(f'fold {fi}')
#   print(f'train {len(train_index)}件:' , np.min(train_index), '~' ,np.max(train_index))
#   print(f'valid {len(valid_index)}件:' , np.min(valid_index), '~' ,np.max(valid_index))
#   print()

In [ ]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from scipy import stats

##こっからテスト

In [ ]:
#作ったモデルのモジュールのテスト
from ubiquantlib.test1models import NN1

In [ ]:


scores = []
y_valid_preds = []
models = []


# 本来はこちら
df_train = train.drop(['investment_id','row_id','time_id','target'],axis=1)
df_target = train['target']
investment_id = train['investment_id']
time_id = train.time_id

#サンプルのとき
# df_train = train_s.drop(['investment_id','row_id','time_id','target'],axis=1)
# df_target = train_s['target']
# investment_id = train_s['investment_id']
# time_id = train_s.time_id


tscv = ubiquantlib.cv.GroupTimeSeriesSplit(n_splits=config['num_kfolds'])
for fi,(train_index, valid_index) in enumerate(tscv.split(df_train, groups=time_id)):
  logger.info('*'*50)
  logger.info(f'fold {fi} start')
  X_train, X_valid = (df_train.iloc[train_index, :], df_train.iloc[valid_index, :])
  y_train, y_valid = (df_target.iloc[train_index], df_target.iloc[valid_index])
  investment_id_train, investment_id_valid = (investment_id.iloc[train_index], investment_id.iloc[valid_index])

  input_size = 300

  #NNの学習
  nn = NN1()

  y_valid_pred, model = nn.train(X_train,
                                 X_valid,
                                 y_train,
                                 y_valid,
                                 config['params'],
                                 investment_id_train,
                                 investment_id_valid,
                                 input_size=input_size,
                                 model_name=f'{BASE_NAME}_{fi}', # 保存先の名前
                                 )
  
  # 結果の保存
  y_valid_preds.append(y_valid_pred)
  models.append(model)

  score = ubiquantlib.evaluate.mean_pearson(train.iloc[valid_index,:].reset_index(drop=True), y_valid_pred)
  scores.append(score)
  logger.info(f'mean pearson: {score}')

  #RAMのために削除
  del  y_valid_pred, model







In [ ]:
# CVスコア
score = sum(scores) / len(scores)
logger.info('===CV scores===')
logger.info(scores)
logger.info(score)

 <module> [INFO]: ===CV scores===
 <module> [INFO]: [0.1285836665992117, 0.08252556664480934, 0.0919727800432226, 0.09073691402985565, 0.11269281153062287]
 <module> [INFO]: 0.10130234776954442


##試行錯誤

In [ ]:
#後で加工するために取り出す
# investment_id = train.pop("investment_id")

# _ = train.pop("time_id")
# _ = train.pop("row_id")
# y = train.pop("target")

In [ ]:
%%time
investment_ids = list(investment_id.unique())
investment_id_size = len(investment_ids) + 1
investment_id_lookup_layer = layers.IntegerLookup(max_tokens=investment_id_size)
investment_id_lookup_layer.adapt(pd.DataFrame({"investment_ids":investment_ids}))

CPU times: user 378 ms, sys: 11.4 ms, total: 389 ms
Wall time: 726 ms


In [ ]:
def preprocess(X, y):
    return X, y
def make_dataset(feature, investment_id, y, batch_size=1024, mode="train"):
    ds = tf.data.Dataset.from_tensor_slices(((investment_id, feature), y))
    ds = ds.map(preprocess)
    if mode == "train":
        ds = ds.shuffle(4096)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [ ]:
train.shape

(100, 300)

In [ ]:
#input層が300列限定のモデルになってる

def get_model():
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse])
    return model

In [ ]:
investment_id.head()

0     244.0
1    3180.0
2    3304.0
3     481.0
4    3346.0
Name: investment_id, dtype: float16

In [ ]:
#array型の型のnp.float32にするとエラーが起きない
encoded_id = investment_id_lookup_layer(investment_id)

train_array = np.asarray(train).astype(np.float32)
id_array = np.asarray(encoded_id).astype(np.float32)
y_array = np.asarray(y).astype(np.float32)

ds = make_dataset(train_array, id_array, y_array)
model = get_model()

In [ ]:
model.fit(ds)

1/1 [==============================] - 2s 2s/step - loss: 7.2493 - mse: 0.7579 - mae: 0.6294 - mape: 143.6452 - rmse: 0.8706


In [ ]:
os.listdir('ubiquantlib')

['datasets.py',
 'cv.py',
 'settings.py',
 'train.py',
 'evaluate.py',
 'models.py',
 'logs.py',
 '__init__.py',
 '__pycache__',
 'test1models.py',
 '.ipynb_checkpoints']